In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import face_recognition
import pickle
import os

In [6]:
def get_name():   # funtion to get name of person
    name=input("Enter your name:")
    return name

In [7]:
root= os.getcwd()
root

'C:\\Users\\SouravKumar\\Desktop\\Face Recognition'

In [8]:
if (os.path.exists("face_images")==0):
    os.mkdir("face_images")
images_folder= os.path.join(root,"face_images")
images_folder

'C:\\Users\\SouravKumar\\Desktop\\Face Recognition\\face_images'

In [9]:
if os.path.exists('known_face_names.txt'):
    with open('known_face_names.txt','rb') as fp:
        known_face_names=pickle.load(fp)
else:
    known_face_names=[]

In [10]:
def cap_img(person_name):  # function to get image of person
    video_capture = cv2.VideoCapture(0)

    cv2.namedWindow("Registering...")

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("failed to grab frame")
            break
        cv2.putText(frame,"Press Space to Capture",(0,20),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,0))
        cv2.imshow("Registering...", frame)

        k = cv2.waitKey(1)
        if k%256== 113:

            print("ending img capture")
            break
        elif k%256 == 32:

            img_name = os.path.join(images_folder ,person_name+".jpg")
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))


    video_capture.release()

    cv2.destroyAllWindows()

In [11]:
def from_img_file(person_name):
    src=input("Enter the path to the image and press enter: ")
    if (os.path.exists==1):
        image= cv2.imread(src)
        img_name = os.path.join(images_folder ,person_name+".jpg")
        cv2.imwrite(img_name, image)
        print("{} written!".format(img_name))
    else:
        print("No image in this path found!")

In [12]:
if os.path.exists('known_face_encodings.txt'):
    with open('known_face_encodings.txt','rb') as fp:
        known_face_encodings=pickle.load(fp)
else:
    known_face_encodings=[]

In [13]:
images={}
encoding={}

In [14]:
def encode(person_name,known_face_encodings):
    images[person_name]=face_recognition.load_image_file(os.path.join(images_folder ,person_name + ".jpg"))
    encoding[person_name]=face_recognition.face_encodings(images[person_name])[0]
    known_face_encodings.append(encoding[person_name])

In [15]:
def recognize_faces(known_face_names, known_face_encodings):
    # Initialize some variables
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    
    # Get a reference to webcam #0 (the default one)
    video_capture = cv2.VideoCapture(0)

    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()

        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # # If a match was found in known_face_encodings, just use the first one.
                # if True in matches:
                #     first_match_index = matches.index(True)
                #     name = known_face_names[first_match_index]

                # Or instead, use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

                face_names.append(name)

        process_this_frame = not process_this_frame


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

In [20]:
while True:
    choice = int(
        input("\n 1.Register yourself into the Database by taking a picture through Webcam \n 2. Register a person through an image of the person \n 3. Webcam Authentication Test \n 4. Exit \n"))
    if (choice == 1):
        person_name=get_name()
        known_face_names.append(person_name)
        cap_img(person_name)
        encode(person_name,known_face_encodings)
    elif (choice==2):
        person_name=get_name()
        known_face_names.append(person_name)
        from_img_file(person_name)
        encode(person_name,known_face_encodings)
    elif (choice == 3):
        recognize_faces(known_face_names, known_face_encodings)
    if (choice >= 4):
        with open('known_face_names.txt', 'wb') as fp:
            pickle.dump(known_face_names,fp)
        with open('known_face_encodings.txt', 'wb') as fp:
            pickle.dump(known_face_encodings,fp)
        break


 1.Register yourself into the Database by taking a picture through Webcam 
 2. Register a person through an image of the person 
 3. Webcam Authentication Test 
 4. Exit 
5
